In [30]:
import os

import pandas as pd
import numpy as np

from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType



In [31]:
REPO_DIRECTORY = r'/Users/kwanshunquinsonhon/Documents/GitHub/'
ABC_DICT_PATH = r'/AIST4010-Cantonese-Translator-Data/ABC-Dict/abc_dict.csv'

def load_abc_dataset():
    abc_dict = pd.read_csv(REPO_DIRECTORY + ABC_DICT_PATH)
    abc_dataset = Dataset.from_pandas(abc_dict)
    return abc_dataset

abc_dict_dataset = load_abc_dataset()
print(abc_dict_dataset[:10])

{'en': ['Scoop up water', 'Ladle out soup', 'Third son of a rich family', 'Young pigeon or squab that has been roasted', "Husband's second older brother", 'This time', 'One dollar and sixty cents', 'Rabbit', 'Exit through turnstile', 'Chop something into cubes'], 'yue': ['㧾水', '㧾湯', '三少', '乳鴿', '二少', '今勻', '個六', '兔仔', '出閘', '切粒']}
